In [2]:
import pandas as pd
import numpy as np
import io
import urllib.request
import matplotlib.pyplot as plt
import os
import gzip
import collections
import re
import json
import xml.etree.ElementTree as ET
import zipfile
import math

%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

For Drugbank Import, download from the website the compressed file, see [here](https://www.drugbank.ca/releases/latest)  
The gzip format works best, so if in *.zip* format, convert to *.gz*

In [5]:
def drugbank_import(path, create_alias='N'):
    
    with gzip.open(path) as xml_file:
        tree = ET.parse(xml_file)
    root = tree.getroot()

    ns = '{http://www.drugbank.ca}'
    calc = "{ns}calculated-properties/{ns}property"
    exp = "{ns}experimental-properties/{ns}property"
    extern = "{ns}external-identifiers/{ns}external-identifier"
    inchikey_template = calc+"[{ns}kind='InChIKey']/{ns}value"
    inchi_template = calc+"[{ns}kind='InChI']/{ns}value"

    melt_point_template = exp+"[{ns}kind='Melting Point']/{ns}value"
    Hydrophobicity_template = exp+"[{ns}kind='Hydrophobicity']/{ns}value"
    isoelectric_template = exp+"[{ns}kind='Isoelectric Point']/{ns}value"
    molweight_template = exp+"[{ns}kind='Molecular Weight']/{ns}value"
    molform_template = exp+"[{ns}kind='Molecular Formula']/{ns}value"
    logP_template = exp+"[{ns}kind='logP']/{ns}value"
    logS_template = exp+"[{ns}kind='logS']/{ns}value"
    
    boil_template = exp+"[{ns}kind='Boiling Point']/{ns}value"
    caco_template = exp+"[{ns}kind='caco2 Permeability']/{ns}value"
    water_exp_template = exp+"[{ns}kind='Water Solubility']/{ns}value"
    pKa_template = exp+"[{ns}kind='pKa']/{ns}value"

    psa_template = calc+"[{ns}kind='Polar Surface Area (PSA)']/{ns}value"
    refr_template = calc+"[{ns}kind='Refractivity']/{ns}value"
    pola_template = calc+"[{ns}kind='Polarizability']/{ns}value"
    bioa_template = calc+"[{ns}kind='Bioavailability']/{ns}value"
    ghose_template = calc+"[{ns}kind='Ghose Filter']/{ns}value"
    mddr_template = calc+"[{ns}kind='MDDR-Like Rule']/{ns}value"
    smiles_template=calc+"[{ns}kind='SMILES']/{ns}value"

    # external identifiers
    DPD_template = extern + \
        "[{ns}resource='Drugs Product Database (DPD)']/{ns}identifier"
    PubChem_template = extern+"[{ns}resource='PubChem Substance']/{ns}identifier"
    kegg_template = extern+"[{ns}resource='KEGG Drug']/{ns}identifier"
    GKB_template = extern+"[{ns}resource='PharmGKB']/{ns}identifier"
    UPKB_template = extern+"[{ns}resource='UniProtKB']/{ns}identifier"
    TTD_template = extern + \
        "[{ns}resource='Therapeutic Targets Database']/{ns}identifier"
    wiki_template = extern+"[{ns}resource='Wikipedia']/{ns}identifier"
    ChEMBL_template = extern+"[{ns}resource='ChEMBL']/{ns}identifier"

    rows = list()
    for i, drug in enumerate(root):
        row = collections.OrderedDict()
        assert drug.tag == ns + 'drug'
        row['type'] = drug.get('type')
        row['drugbank_id'] = drug.findtext(ns + "drugbank-id[@primary='true']")
        row['average-mass'] = drug.findtext(ns + "average-mass")
        row['monoisotopic-mass'] = drug.findtext(ns + "monoisotopic-mass")
        row['name'] = drug.findtext(ns + "name")
        # free text
        row['volume-of-distribution'] = drug.findtext(
            ns + "volume-of-distribution")
        row['clearance'] = drug.findtext(ns + "clearance")
        row['half-life'] = drug.findtext(ns + "half-life")
        row['toxicity'] = drug.findtext(ns + "toxicity")
        row['metabolism'] = drug.findtext(ns + "metabolism")
        row['absorption'] = drug.findtext(ns + "absorption")
        
        row['smiles'] = drug.findtext(smiles_template.format(ns=ns))
        # experimental
        row['melting point'] = drug.findtext(melt_point_template.format(ns=ns))
        row['Hydrophobicity'] = drug.findtext(
            Hydrophobicity_template.format(ns=ns))
        row['Isoelectric Point'] = drug.findtext(
            isoelectric_template.format(ns=ns))
        row['Molecular Weight'] = drug.findtext(molweight_template.format(ns=ns))
        row['Molecular Formula'] = drug.findtext(molform_template.format(ns=ns))
        row['logP EXP'] = drug.findtext(logS_template.format(ns=ns))
        row['logS EXP'] = drug.findtext(melt_point_template.format(ns=ns))
        row['pKa EXP'] = drug.findtext(pKa_template.format(ns=ns))
        row['Boiling Point'] = drug.findtext(boil_template.format(ns=ns))
        row['Caco2 Permeability'] = drug.findtext(caco_template.format(ns=ns))
        row['Water Solubility EXP'] = drug.findtext(
            water_exp_template.format(ns=ns))
        # calculated
        row['PSA calc'] = drug.findtext(psa_template.format(ns=ns))
        row['Refractivity calc'] = drug.findtext(refr_template.format(ns=ns))
        row['Polarizability'] = drug.findtext(pola_template.format(ns=ns))
        row['Bioavailability'] = drug.findtext(bioa_template.format(ns=ns))
        row['Ghose Filter'] = drug.findtext(ghose_template.format(ns=ns))
        row['MDDR-Like Rule'] = drug.findtext(mddr_template.format(ns=ns))
        # external
        row['Drugs Product Database (DPD)'] = drug.findtext(
            DPD_template.format(ns=ns))
        row['PubChem Substance'] = drug.findtext(PubChem_template.format(ns=ns))
        row['KEGG Drug'] = drug.findtext(kegg_template.format(ns=ns))
        row['PharmGKB'] = drug.findtext(GKB_template.format(ns=ns))
        row['UniProtKB'] = drug.findtext(UPKB_template.format(ns=ns))
        row['Therapeutic Targets Database'] = drug.findtext(
            TTD_template.format(ns=ns))
        row['Wikipedia'] = drug.findtext(wiki_template.format(ns=ns))
        row['ChEMBL'] = drug.findtext(ChEMBL_template.format(ns=ns))

        # others
        row['groups'] = [group.text for group in
                         drug.findall("{ns}groups/{ns}group".format(ns=ns))]
        row['atc_codes'] = [code.get('code') for code in
                            drug.findall("{ns}atc-codes/{ns}atc-code".format(ns=ns))]
        row['categories'] = [x.findtext(ns + 'category') for x in
                             drug.findall("{ns}categories/{ns}category".format(ns=ns))]
        row['inchi'] = drug.findtext(inchi_template.format(ns=ns))
        row['inchikey'] = drug.findtext(inchikey_template.format(ns=ns))
        
        # Add drug aliases
        aliases = {
            elem.text for elem in
            drug.findall("{ns}international-brands/{ns}international-brand".format(ns=ns)) +
            drug.findall("{ns}synonyms/{ns}synonym[@language='English']".format(ns=ns)) +
            drug.findall("{ns}international-brands/{ns}international-brand".format(ns=ns)) +
            drug.findall("{ns}products/{ns}product/{ns}name".format(ns=ns))

        }
        aliases.add(row['name'])
        row['aliases'] = sorted(aliases)

        rows.append(row)
        
    if create_alias=='Y':
        alias_dict = {row['drugbank_id']: row['aliases'] for row in rows}
        with open('aliases.json', 'w') as fp:
            json.dump(alias_dict, fp, indent=2, sort_keys=True)
        


    def collapse_list_values(row):
        for key, value in row.items():
            if isinstance(value, list):
                row[key] = '|'.join(value)
        return row


    rows = list(map(collapse_list_values, rows))
    print(rows)
    columns = ['drugbank_id', 'name', 'type', 'groups', 'atc_codes',
               'categories', 'inchikey', 'inchi', 'average-mass',
               'monoisotopic-mass', 'volume-of-distribution', 'clearance', 'half-life',
               'toxicity', 'metabolism', 'metabolism', 'absorption', 'smiles',
               'melting point', 'logS EXP', 'logP EXP', 'pKa EXP', 'Isoelectric Point', 'Molecular Weight', 'Molecular Formula',
               'Hydrophobicity', 'Boiling Point', 'Caco2 Permeability', 'Water Solubility EXP', 'PSA calc', 'Refractivity calc', 'Polarizability', 'Ghose Filter', 'MDDR-Like Rule',
               'Drugs Product Database (DPD)', 'PubChem Substance', 'KEGG Drug', 'PharmGKB', 'UniProtKB', 'Therapeutic Targets Database', 'ChEMBL', 'Wikipedia']

    drugbank_df = pd.DataFrame.from_dict(rows)[columns]

    drugbank_slim_df = drugbank_df[
        drugbank_df.groups.map(lambda x: 'approved' in x) &
        drugbank_df.inchi.map(lambda x: x is not None) &
        drugbank_df.type.map(lambda x: x == 'small molecule')
    ]

    return drugbank_df,drugbank_slim_df

In [ ]:
dbdata,dbdataslim=drugbank_import('../files/full database.xml.gz')

In [ ]:
dbdata.head()
dbdata.describe()

In [ ]:
dbdata['average-mass']=pd.to_numeric(dbdata['average-mass'], errors='coerce')
dbdata['monoisotopic-mass']=pd.to_numeric(dbdata['monoisotopic-mass'], errors='coerce')
dbdata['logP EXP'].describe()

In [ ]:
dbdata.to_csv('drugbank.csv',index=False,quoting=1)
#quoting for then be able to successfully import into tableau or ELK